In [ ]:
import numpy as np
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from Plots import plot_action_profit_percent
from Dataset import BitcoinData
from TradingEnvironment import TradingEnvironment
from Models import DoubleDQN

from keras.models import load_model

tf.keras.utils.disable_interactive_logging()

NUM_EPISODES = 1
NUM_EPISODES_TEST = 1

MODEL_DQN = 0
MODEL_DOUBLE_DQN = 1
MODEL_DUELING_DQN = 2

In [ ]:
def get_environment(start_time=0):
    bitcoin_data = BitcoinData()

    return TradingEnvironment(bitcoin_data, start_time=start_time)

def get_agent(environment, model_type, load_version=-1, load_model_num=-1):
    observation_space = environment.get_observation_space()
    action_space = environment.get_action_space()

    if model_type == MODEL_DOUBLE_DQN:
        return DoubleDQN(observation_space, action_space, model_path="DoubleDQN", 
                         load_version=load_version, load_model_num=load_model_num)    
    return None

def cryptocurrencies_trading(agent, environment, train=True):
    observation_space = environment.get_observation_space()
    episode = 0

    while episode < NUM_EPISODES:
        episode += 1
        state = environment.reset()
        step = 0
        while True:
            step += 1
            action = agent.act(np.reshape(state, [1, observation_space]))
            next_state, reward, done, info = environment.step(action - 1)

            if train:
                agent.memory.remember(state, action, reward, next_state, done)
                agent.learn()

            state = next_state

            if (step - 1) % 100 == 0:
                print("Step {}:".format(step - 1))
                print("    Percent Capital: {:.4f}, interval: [{:.4f}, {:.4f}]".format(
                    environment.percent_profit, environment.min_percent_profit, environment.max_percent_profit
                ))
                print("    Action taked by itseft: SELL: {}, HOLD: {}, BUY: {}".format(
                    agent.actions_taked_by_itself[0], agent.actions_taked_by_itself[1], agent.actions_taked_by_itself[2]
                ))

                agent.actions_taked_by_itself = [0, 0, 0]

            if done:
                print("Episode {}/{} ended\n", episode, NUM_EPISODES)
                break

        if train: 
            agent.save_model()


In [ ]:
environment = get_environment()
agent = get_agent(environment, MODEL_DOUBLE_DQN)

cryptocurrencies_trading(agent, environment)

In [ ]:
last_episode = 2
start_time = TradingEnvironment.NUM_OBSERVATIONS * last_episode
load_version = 7
load_model_num = -1

environment = get_environment(start_time=start_time)
agent = get_agent(environment, MODEL_DOUBLE_DQN, load_version=load_version, load_model_num=load_model_num)
agent.exploration_rate = 0.0

#cryptocurrencies_trading(agent, environment, train=False)

In [ ]:
start_time = 0
end_time = start_time + 1000
plot_action_profit_percent(agent.memory.states, agent.memory.actions, agent.memory.rewards, agent.memory.dones, start_time=start_time, end_time=end_time)

In [ ]:
agent = DoubleDQN(environment.get_observation_space(), environment.get_action_space(), model_path="DoubleDQN")

In [ ]:
state = environment.reset()
for i in range(0, 100):
    next_state, reward, done, info = environment.step(0)
    agent.memory.remember(state, 0, reward, next_state, done)
    state = next_state



In [ ]:
path_to_memory = os.path.join(agent.model_path, agent.MEMORY_FILENAME)
agent.save_model()

In [ ]:
np.set_printoptions(suppress=True)
agent.memory.states[0:55]

In [ ]:
np.random.choice(5, 3, replace=False)

In [ ]:
environment.wallet